In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

# Assigned data directly to lists
encoder_input_data = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                               [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                               [21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
                               [31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
                               [41, 42, 43, 44, 45, 46, 47, 48, 49, 50]])

decoder_input_data = np.array([[51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
                               [61, 62, 63, 64, 65, 66, 67, 68, 69, 70],
                               [71, 72, 73, 74, 75, 76, 77, 78, 79, 80],
                               [81, 82, 83, 84, 85, 86, 87, 88, 89, 90],
                               [91, 92, 93, 94, 95, 96, 97, 98, 99, 100]])

decoder_target_data = np.roll(decoder_input_data, shift=-1, axis=1)
decoder_target_data[:, -1] = 0

# Seq2Seq Model
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=101, output_dim=latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=101, output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(101, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=10,
          validation_split=0.2)

# Inference Setup

# Encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Function to decode sequences
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = 1

    decoded_sentence = []
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sentence.append(sampled_token_index)
        if sampled_token_index == 0 or len(decoded_sentence) > 10:
            stop_condition = True
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence

# Test
print("Encoded sentence:", encoder_input_data[0:1])
test_input = encoder_input_data[0:1]
decoded_sentence = decode_sequence(test_input)
print('Decoded sentence:', decoded_sentence)


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_17 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 input_18 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_10 (Embedding)    (None, None, 256)            25856     ['input_17[0][0]']            
                                                                                                  
 embedding_11 (Embedding)    (None, None, 256)            25856     ['input_18[0][0]']            
                                                                                           